In [1]:
from langchain_community.document_loaders import YoutubeLoader
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown, Latex
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from crewai_tools import BaseTool

load_dotenv()

model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4o-mini")
llm = ChatOpenAI(temperature=0, model_name=model_name)

In [2]:
import urllib
import warnings

from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

warnings.filterwarnings("ignore")

class SumarizeVideo(BaseTool):
    name: str = "Resumir Vídeo"
    description: str = (
        "Resume a transcrição de um vídeo do YouTube."
    )

    def _run(self, url_video: str) -> str:
        llm = ChatOpenAI(temperature=0, model_name=model_name)

        loader = YoutubeLoader.from_youtube_url(
            url_video,
            add_video_info=True,
            language=["pt","en"],
        )
        document = loader.load()

        question_prompt_template = """
                        Forneça um resumo em tópicos dos assuntos tratados no texto a seguir.
                        TEXTO: {text}
                        RESUMO:
                        """

        question_prompt = PromptTemplate(
            template=question_prompt_template, input_variables=["text"]
        )

        refine_prompt_template = """
                    Escreva um resumo conciso do texto a seguir delimitado por crases triplas.
                    Retorne sua resposta em tópicos que cubram todos os pontos-chave do texto.
                    ```{text}```
                    RESUMO DOS PONTOS:
                    """

        refine_prompt = PromptTemplate(
            template=refine_prompt_template, input_variables=["text"]
        )

        refine_chain = load_summarize_chain(
            llm,
            chain_type="refine",
            question_prompt=question_prompt,
            refine_prompt=refine_prompt,
            return_intermediate_steps=True,
        )
        refine_outputs = refine_chain({"input_documents": document})

        return refine_outputs["output_text"]


from embedchain import App
import ast


class SearchTranscript(BaseTool):
    name: str = "Busca em Transcrição"
    description: str = (
        "Busca o tópico nos vídeos do YouTube e retorna os dados."
    )

    def _run(self, url_video: str, seleted_topic: str) -> str:
        app = App.from_config(config_path="youtube_video.yaml")
        app.add(url_video, data_type="youtube_video")
        results = app.search(f"O que é dito no vídeo sobre: {seleted_topic}", where={"url": url_video})
        
        output = ""
        output += f"### Tópico: {seleted_topic}\n\n"
        for i in range(len(results)):
            transcript = ast.literal_eval(results[i]['metadata']['transcript'])
            context = results[i]['context']
            output += f"**Contexto:** {context}\n\n"
            # iterar pelo transcript e verificar se o text está no context
            for chunk in transcript:
                if chunk['text'] in context[:70]:
                    segundos = f"{str(chunk['start'])}s"
                    title = results[i]['metadata']['title'] + " - " + segundos
                    output += f"**Abra o vídeo neste ponto:** [{title}](https://www.youtube.com/watch?v=VsZXAEFKXC4&t={segundos})\n\n"
        
        return output
       

       

In [3]:
from crewai import Agent, Crew, Process, Task

# abrir arquivo yaml como dict
def load_yaml(file_path):
    import yaml
    with open(file_path, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

agents_config = load_yaml("src/youtube_sommelier/config/agents.yaml")
tasks_config = load_yaml("src/youtube_sommelier/config/tasks.yaml")

In [4]:
video_sumarizer = Agent(
	config=agents_config['video_sumarizer'],
    tools=[SumarizeVideo()],
    verbose=True,
    llm=llm,
	allow_delegation=False,
)

sumarize_video = Task(
	config=tasks_config['sumarize_video'],
    agent=video_sumarizer,
    human_input=True
)

video_searcher = Agent(
    config=agents_config['video_searcher'],
    tools=[SearchTranscript(result_as_answer=True)],
    verbose=True,
    llm=llm,
    allow_delegation=False,
)

search_video = Task(
    config=tasks_config['search_video'],
    agent=video_searcher
)

crew = Crew(
  agents=[video_sumarizer, video_searcher],
  tasks=[sumarize_video, search_video],
  verbose=2,
  memory=False,
  llm=llm,
  full_output=True,
  planning=False,
)

input_text  = "https://www.youtube.com/watch?v=VsZXAEFKXC4"

inputs = {
    "topic": input_text
}
result = crew.kickoff(inputs=inputs)

 [2024-08-01 23:58:35][DEBUG]: == Working Agent: Sumarizador de vídeo

 [2024-08-01 23:58:35][INFO]: == Starting Task: Entrada: https://www.youtube.com/watch?v=VsZXAEFKXC4 Verifique se na entrada informada existe um vídeo. Se sim, resuma o conteúdo do vídeo em tópicos principais. Com base no retorno do humano, apenas informe o tópico selecionado.



> Entering new CrewAgentExecutor chain...
Action: Resumir Vídeo  
Action Input: {"url_video":"https://www.youtube.com/watch?v=VsZXAEFKXC4"}   

**Resumo em Tópicos:**

1. **Introdução ao Vídeo:**
   - O apresentador busca uma polêmica para atrair visualizações.
   - O cenário é uma clínica de fisioterapia, onde ocorre uma entrevista enquanto se treina.

2. **Entrevista com Julinho da Purples:**
   - Julinho é um músico conhecido e fala sobre sua trajetória na música gospel.
   - Ele compartilha sua conversão e o início de seu envolvimento com a música na igreja.

3. **Experiência Musical:**
   - Julinho começou tocando bateria e, posteriorm

2024-08-01 23:59:54,462 - 138910512784448 - add_config.py-add_config:30 - WARNING: min_chunk_size 0 should be greater than chunk_overlap 100, otherwise it is redundant.


 

### Tópico: Julinho começou tocando bateria e, posteriormente, passou a cantar.

**Contexto:** é uma loucura carao exatamente mas é o que eu ia falar você falou de chamado é dá vontade de se trancar numa caverna mas ao mesmo tempo Deus te mandou fazer uma coisa e você tá obedecendo É isso mesmo se você é bom nisso é porque Deus te vocacion então eu quero continuar obedecendo mano É isso mesmo vou parar porque a minha limitação e vou fazer se você gostou do vídeo de hoje que foi Júnior da pur muito um treino muito pesado né morremos aqui Então se inscreva no canal Julio da purs como encontrá-lo nas redes sociais @ dois dedos de TO tamb vai lá mas on dia é que a gente encontra ô Júlio Filho você me encontra na minha rede social Pessoal e e purples Banda né um pouco mais sobre a purples purpl Banda vai tá escrito aqui para você ir atrás n redes sociais aí ou seas canções são bênção pra vida da [Música] igreja

**Abra o vídeo neste ponto:** [COBRAR CACHÊ? MÚSICO É TROUXA? VICIADOS EM AG

In [5]:
Markdown(result.raw)

### Tópico: Julinho começou tocando bateria e, posteriormente, passou a cantar.

**Contexto:** é uma loucura carao exatamente mas é o que eu ia falar você falou de chamado é dá vontade de se trancar numa caverna mas ao mesmo tempo Deus te mandou fazer uma coisa e você tá obedecendo É isso mesmo se você é bom nisso é porque Deus te vocacion então eu quero continuar obedecendo mano É isso mesmo vou parar porque a minha limitação e vou fazer se você gostou do vídeo de hoje que foi Júnior da pur muito um treino muito pesado né morremos aqui Então se inscreva no canal Julio da purs como encontrá-lo nas redes sociais @ dois dedos de TO tamb vai lá mas on dia é que a gente encontra ô Júlio Filho você me encontra na minha rede social Pessoal e e purples Banda né um pouco mais sobre a purples purpl Banda vai tá escrito aqui para você ir atrás n redes sociais aí ou seas canções são bênção pra vida da [Música] igreja

**Abra o vídeo neste ponto:** [COBRAR CACHÊ? MÚSICO É TROUXA? VICIADOS EM AGRADAR? | Fé em Forma com Julinho da Purples - 1586.159s](https://www.youtube.com/watch?v=VsZXAEFKXC4&t=1586.159s)

**Contexto:** sabem que eu o que eu faço é isso então ao mesmo tempo que eu recebo encorajamento eu recebo muita exortação e muita muita correção da parte dos meus pais da parte dos meus amigos mais próximos e isso eu acho que é o maior tesouro que a gente tem como igreja e como pessoas e submetidas à visibilidade é é você não e se apoiar nos seus números porque aquilo lá é é tapinha nas costas de quem não te conhece de quem não tá te vendo na igreja ou de quem tipo só achou legal o jeito que você canta o jeito que você prega isso não quer dizer nada nada nada isso não tem profundidade nenhuma Isso é uma extensão o espírito santo decidiu levar o seu ministério e não tem nada a ver com você você imaginava que ia chegar a tantos números porque não é você cara entendeu Deus te vocacion para um trabalho e ele escolheu te usar dessa forma mas não é sobre você é porque qualquer hora pode ser acontecer alguma coisa ou ou simplesmente você falecer é e a mensagem vai continuar sendo pregada porque Deus não precisa de nós você arrumar tua mulher amanhã entendeu se eu cair na droga amanhã entendeu se eu pular da ponte amanhã se eu roubar amanhã E aí entendeu então é é eu acho que o principal da gente eh se manter nos trilhos Além da Vida devocional e você se alimentar da palavra sim é você se cercar de pessoas de novo repetir que não estão impressionadas pelo que você faz é muito gostos cara esse conselho vi do M muito então é muito gostoso Fala aí é gostoso quando as pessoas estão impressionadas pelo que você faz a gente se sente útil a gente se s é é especial especial mas ao mesmo tempo você vai dormir no outro dia você acordou passou é é é Acaba vendade entendeu é muito doido cara quando você assim a melhor coisa do mundo é o fato de você não lembrar que você é alguém além daquilo que você é no dia a dia então quem é que a gente é no dia a dia eu sou um pai cuidando dos filhos igual qualquer pai sou um marido cuidando da esposa igual qualquer marido sou um profissional Tentando ganhar a

**Abra o vídeo neste ponto:** [COBRAR CACHÊ? MÚSICO É TROUXA? VICIADOS EM AGRADAR? | Fé em Forma com Julinho da Purples - 1378.72s](https://www.youtube.com/watch?v=VsZXAEFKXC4&t=1378.72s)

**Abra o vídeo neste ponto:** [COBRAR CACHÊ? MÚSICO É TROUXA? VICIADOS EM AGRADAR? | Fé em Forma com Julinho da Purples - 1381.799s](https://www.youtube.com/watch?v=VsZXAEFKXC4&t=1381.799s)

**Contexto:** voto é o rap das eleições né do nosso voto nos declaração de voto E aí e eu fiz uma música de Natal cara eu toco violão e canto foi agora agora fim de ano em que eu toco violão e canta mú mas eu eu eu eu gosto muito de Natal mano eu gosto de Natal Cara eu amo Natal amo Natal e por isso que eu que eu fiz uma música de Natal cara e ficou muito bonitinha a música de natal é mesmo que que eu queria eu queria botar a galera da igreja para cantar e pronto porque eu sei eu sou eu acanho ah pra igreja cantar mesmo é é Congregacional cara é Congregacional a gente cantou no C de Natal da Igreja e assim eu queria colocar para o pessoal que tocou eles sozinhos e pronto e botar lá no YouTube mas se eu jogo uma música no YouTube só com o pessoal da minha igreja cantando um monte de gente que ninguém sabe quem é não vai clicar para ver aí eu tinha que me aí eu fiz o quê Eu Aprendi a tocar quatro notas no violão e aí toquei quatro notas no violão na música lá atrin lá no o quinto violão e fiz lá uma a segunda voz no fundinho bem baixinho que deu um trabalho do cão para gravar foi o músico o vocalista lá da igreja o regente do coral pro estúdio ele cantava uma frase no Tom a eu tentava imitar tentava imitar aí quando eu tava imitando direitinho eu cantava no microfone ele não t horrível Faz de novo aía palavra por palavra tentando acertar quando eu fiz o mais parecido possível aí foram afinar minha voz lá no tá vendo o que que é pessoa no ministério certo não sou músico não sou músico é isso eu não sou músico Mas ó quantas pessoas f de um assunto polêmica ponto é eu precisava fazer isso PR eu aparecer no vídeo só para poder assim ó o Iago tá tocando violão eu vou clicar para assistir e aí as pessoas verem a música mas euv um Quad vante porque eu apareço fazendo o primeiro blim BL BL e depois meu irmão é só o pessoal lá entendeu só por mim eu nem aparecia na mas quanta gente na igreja por causa de melindre e maturidade canta mal mas ninguém tem coragem de chegar para ele e falar

**Abra o vídeo neste ponto:** [COBRAR CACHÊ? MÚSICO É TROUXA? VICIADOS EM AGRADAR? | Fé em Forma com Julinho da Purples - 1181.76s](https://www.youtube.com/watch?v=VsZXAEFKXC4&t=1181.76s)

